In [3]:
import sys
sys.path.append('/home/mourad/DS/Mosmovi/Mosmovi/')
import pandas as pd
import os
import numpy as np
import multiprocessing as mp
import seaborn as sns
import matplotlib.pyplot as plt
import random

import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

pd.set_option('max_columns', None)
pd.set_option('display.max_rows', 100)
from collections import Counter

import unicodedata
import importlib

from sklearn.model_selection import train_test_split

pd.set_option('display.float_format', lambda x: '%.18f' % x)

In [4]:
def format_csvs_to_df(path):
    file_names = os.listdir(path)
    frames = []
    for n in file_names:
        coordinates = [float(x) for x in n[:-4].split('_')]
        df = pd.read_csv(path + n, sep=';')
        df['coordinates'] = [coordinates] * len(df)
        frames.append(df)
    df_total = pd.concat(frames)
    return df_total

def train_test_split_by_users(df, percentage_users = 0.10):
    """
    Take a dataframe in input and return train and test sets. 
    """
    users_ids = df.author_id
    users_ids = set(df.author_id)
    n_samples = int(len(users_ids)*percentage_users)
    val_users = random.sample(users_ids, n_samples)
    ind = df.author_id.isin(val_users)
    return df.loc[~ind], df.loc[ind]

In [5]:
%%time
df = format_csvs_to_df('../../data/Copy of dev_south_america_merged/')

CPU times: user 1min 36s, sys: 6.31 s, total: 1min 42s
Wall time: 1min 44s


In [6]:
%%time
#Convert to datetime
df['created_at'] = pd.to_datetime(df['created_at'])
#Extract lat and long
df['lat'], df['long'] = df.coordinates.apply(lambda x : x[0]), df.coordinates.apply(lambda x : x[1])
#Avoid scientific notation for ID type
df['id'] = df['id'].astype(np.int64).astype(str)
df['conversation_id'] = df['conversation_id'].fillna(0).astype(np.int64).astype(str)
df['in_reply_to_user_id'] = df['in_reply_to_user_id'].fillna(0).astype(np.int64).astype(str)
df['author_id'] = df['author_id'].fillna(0).astype(np.int64).astype(str)

CPU times: user 30.7 s, sys: 2.14 s, total: 32.9 s
Wall time: 32.9 s


### Duplicates